In [110]:
import pandas as pd
import scipy as sc
import numpy as np
import os


import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import colorConverter
import seaborn as sns

%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [111]:
from constants import guest_types
from constants import colors
from bootstrap import bootstrap
from bootstrap import dG_bootstrap
from bootstrap import dH_bootstrap
import plotting

In [112]:
from constants import experimental_deltaG
from constants import experimental_deltaH
from constants import systems

In [113]:
from summarize_statistics import write_statistics

# Calculate or load in the summary results

In [108]:
bgbg = pd.read_csv("results/bgbg_tip3p_by_orientation.csv")
bgbg_combined = pd.read_csv("results/bgbg_tip3p_combined.csv")

bg2bg2 = pd.read_csv("results/bg2bg2_tip3p_by_orientation.csv")
bg2bg2_combined = pd.read_csv("results/bg2bg2_tip3p_combined.csv")

smirnoff = pd.read_csv("results/smirnoff_by_orientation.csv")
smirnoff_combined = pd.read_csv("results/smirnoff_combined.csv")

experimental = pd.read_csv("results/experimental.csv")

In [ ]:
data_sets = [bgbg_combined, bg2bg2_combined, smirnoff_combined, experimental]
names = ["GAFF v1.7", "GAFF v2.1", "SMIRNOFF99Frosst", "Experimental"]

In [ ]:
from itertools import permutations

In [33]:
delta_G_r_squared = pd.DataFrame()
delta_G_rmse = pd.DataFrame()
delta_G_mse = pd.DataFrame()

delta_H_r_squared = pd.DataFrame()
delta_H_rmse = pd.DataFrame()
delta_H_mse = pd.DataFrame()

for index_i, i in enumerate(data_sets):
    for index_j, j in enumerate(data_sets):
                    
        # This seems the most reliable way of matching keys. Took a little while to figure out,
        # even if the data frames are ordered properly.
        df = i.merge(j, on=["System"], suffixes=("_i", "_j"))    
        
        results = bootstrap(df["Delta G_i"], df["G_SEM_i"], df["Delta G_j"], df["G_SEM_j"], cycles=1000)
        print(f"{names[index_i]} -- {names[index_j]} {results['mean']['R**2']}")


        delta_G_r_squared = delta_G_r_squared.append(
            {
                "x": names[index_i],
                "y": names[index_j],
                "R**2": results["mean"]["R**2"],
                "R**2 SEM": results["sem"]["R**2"]
            },
            ignore_index=True,
        )

        delta_G_rmse = delta_G_rmse.append(
            {
                "x": names[index_i],
                "y": names[index_j],
                "RMSE": results["mean"]["RMSE"],
                "RMSE SEM": results["sem"]["RMSE"]
            },
            ignore_index=True,
        )

        delta_G_mse = delta_G_mse.append(
            {
                "x": names[index_i],
                "y": names[index_j],
                "MSE": results["mean"]["MSE"],
                "MSE SEM": results["mean"]["MSE"]
            },
            ignore_index=True,
        )
        
        results = bootstrap(df["Delta H_i"], df["H_SEM_i"], df["Delta H_j"], df["H_SEM_j"], cycles=1000)
        print(f"{names[index_i]} -- {names[index_j]} {results['mean']['R**2']}")
        
        delta_H_r_squared = delta_H_r_squared.append(
            {
                "x": names[index_i],
                "y": names[index_j],
                "R**2": results["mean"]["R**2"],
                "R**2 SEM": results["sem"]["R**2"]
                
            },
            ignore_index=True,
        )

        delta_H_rmse = delta_H_rmse.append(
            {
                "x": names[index_i],
                "y": names[index_j],
                "RMSE": results["mean"]["RMSE"],
                "RMSE SEM": results["sem"]["RMSE"]
            },
            ignore_index=True,
        )

        delta_H_mse = delta_H_mse.append(
            {
                "x": names[index_i],
                "y": names[index_j],
                "MSE": results["mean"]["MSE"],
                "MSE SEM": results["sem"]["MSE"]
            },
            ignore_index=True,
        )


GAFF v1.7 -- GAFF v1.7 0.8640459088400202
GAFF v1.7 -- GAFF v1.7 0.8952935292070222
GAFF v1.7 -- GAFF v2.1 0.5443148855533857
GAFF v1.7 -- GAFF v2.1 0.6117083425882566
GAFF v1.7 -- SMIRNOFF99Frosst 0.5856835226983856
GAFF v1.7 -- SMIRNOFF99Frosst 0.6944846702335524
GAFF v1.7 -- Experimental 0.5370083937757651
GAFF v1.7 -- Experimental 0.3936289653837588
GAFF v2.1 -- GAFF v1.7 0.5377748892540649
GAFF v2.1 -- GAFF v1.7 0.6091330657408875
GAFF v2.1 -- GAFF v2.1 0.9574387696819072
GAFF v2.1 -- GAFF v2.1 0.974101682963238
GAFF v2.1 -- SMIRNOFF99Frosst 0.3711021040197053
GAFF v2.1 -- SMIRNOFF99Frosst 0.5979700428934158
GAFF v2.1 -- Experimental 0.8225389382298827
GAFF v2.1 -- Experimental 0.7507165075495107
SMIRNOFF99Frosst -- GAFF v1.7 0.5913937963590066
SMIRNOFF99Frosst -- GAFF v1.7 0.6967604030590443
SMIRNOFF99Frosst -- GAFF v2.1 0.36670847384122957
SMIRNOFF99Frosst -- GAFF v2.1 0.6040172606566303
SMIRNOFF99Frosst -- SMIRNOFF99Frosst 0.7451402000029268
SMIRNOFF99Frosst -- SMIRNOFF99Frosst

In [8]:
delta_G_r_squared_pivot = delta_G_r_squared.pivot(columns="y", index="x")
delta_G_rmse_pivot = delta_G_rmse.pivot(columns="y", index="x",)
delta_G_mse_pivot = delta_G_mse.pivot(columns="y", index="x",)

delta_H_r_squared_pivot = delta_H_r_squared.pivot(columns="y", index="x", )
delta_H_rmse_pivot = delta_H_rmse.pivot(columns="y", index="x",)
delta_H_mse_pivot = delta_H_mse.pivot(columns="y", index="x", )


In [9]:
pd.options.display.float_format = '{:,.2f}'.format
delta_G_r_squared_pivot

R**2                                       \
y                Experimental GAFF v1.7 GAFF v2.1 SMIRNOFF99Frosst   
x                                                                    
Experimental             0.94      0.53      0.82             0.34   
GAFF v1.7                0.53      0.86      0.54             0.59   
GAFF v2.1                0.82      0.54      0.96             0.37   
SMIRNOFF99Frosst         0.34      0.59      0.37             0.74   

                     R**2 SEM                                       
y                Experimental GAFF v1.7 GAFF v2.1 SMIRNOFF99Frosst  
x                                                                   
Experimental             0.08      0.10      0.08             0.11  
GAFF v1.7                0.10      0.04      0.10             0.11  
GAFF v2.1                0.08      0.10      0.01             0.11  
SMIRNOFF99Frosst         0.11      0.12      0.11             0.09

In [10]:
delta_G_rmse_pivot

RMSE                                       \
y                Experimental GAFF v1.7 GAFF v2.1 SMIRNOFF99Frosst   
x                                                                    
Experimental             0.21      0.88      1.68             0.91   
GAFF v1.7                0.88      0.37      2.22             0.80   
GAFF v2.1                1.68      2.23      0.29             1.89   
SMIRNOFF99Frosst         0.91      0.79      1.90             0.46   

                     RMSE SEM                                       
y                Experimental GAFF v1.7 GAFF v2.1 SMIRNOFF99Frosst  
x                                                                   
Experimental             0.16      0.09      0.09             0.10  
GAFF v1.7                0.09      0.04      0.12             0.12  
GAFF v2.1                0.09      0.12      0.03             0.13  
SMIRNOFF99Frosst         0.11      0.11      0.14             0.07

In [11]:
delta_G_mse_pivot

MSE                                       \
y                Experimental GAFF v1.7 GAFF v2.1 SMIRNOFF99Frosst   
x                                                                    
Experimental             0.00      0.46     -1.56            -0.01   
GAFF v1.7               -0.46     -0.00     -2.01            -0.47   
GAFF v2.1                1.56      2.02     -0.00             1.54   
SMIRNOFF99Frosst         0.01      0.47     -1.54             0.00   

                      MSE SEM                                       
y                Experimental GAFF v1.7 GAFF v2.1 SMIRNOFF99Frosst  
x                                                                   
Experimental             0.00      0.46     -1.56            -0.01  
GAFF v1.7               -0.46     -0.00     -2.01            -0.47  
GAFF v2.1                1.56      2.02     -0.00             1.54  
SMIRNOFF99Frosst         0.01      0.47     -1.54             0.00

In [12]:
delta_H_r_squared_pivot

R**2                                       \
y                Experimental GAFF v1.7 GAFF v2.1 SMIRNOFF99Frosst   
x                                                                    
Experimental             0.99      0.39      0.75             0.44   
GAFF v1.7                0.39      0.89      0.61             0.70   
GAFF v2.1                0.75      0.61      0.97             0.60   
SMIRNOFF99Frosst         0.44      0.70      0.59             0.89   

                     R**2 SEM                                       
y                Experimental GAFF v1.7 GAFF v2.1 SMIRNOFF99Frosst  
x                                                                   
Experimental             0.01      0.12      0.08             0.12  
GAFF v1.7                0.12      0.04      0.10             0.09  
GAFF v2.1                0.08      0.10      0.01             0.10  
SMIRNOFF99Frosst         0.11      0.09      0.10             0.04

In [13]:
delta_H_rmse_pivot

RMSE                                       \
y                Experimental GAFF v1.7 GAFF v2.1 SMIRNOFF99Frosst   
x                                                                    
Experimental             0.15      2.55      2.21             1.85   
GAFF v1.7                2.54      0.71      3.88             1.65   
GAFF v2.1                2.20      3.88      0.44             2.96   
SMIRNOFF99Frosst         1.86      1.65      2.97             0.75   

                     RMSE SEM                                       
y                Experimental GAFF v1.7 GAFF v2.1 SMIRNOFF99Frosst  
x                                                                   
Experimental             0.06      0.24      0.22             0.23  
GAFF v1.7                0.24      0.11      0.21             0.19  
GAFF v2.1                0.23      0.22      0.05             0.23  
SMIRNOFF99Frosst         0.22      0.17      0.23             0.12

In [14]:
delta_H_mse_pivot

MSE                                       \
y                Experimental GAFF v1.7 GAFF v2.1 SMIRNOFF99Frosst   
x                                                                    
Experimental             0.00      1.85     -1.64             0.77   
GAFF v1.7               -1.84     -0.00     -3.48            -1.08   
GAFF v2.1                1.63      3.48      0.00             2.40   
SMIRNOFF99Frosst        -0.77      1.08     -2.41            -0.00   

                      MSE SEM                                       
y                Experimental GAFF v1.7 GAFF v2.1 SMIRNOFF99Frosst  
x                                                                   
Experimental             0.03      0.27      0.22             0.27  
GAFF v1.7                0.27      0.11      0.26             0.20  
GAFF v2.1                0.23      0.27      0.07             0.27  
SMIRNOFF99Frosst         0.26      0.19      0.27             0.11

In [10]:
def pretty_print(results):
    print(f"{results['mean']['RMSE']:2.2f} ± {results['sem']['RMSE']:2.2f}", end="")
    print("\n", end="")
    print(f"{results['mean']['MSE']:2.2f} ± {results['sem']['MSE']:2.2f}", end="")
    print("\n", end="")
    print(f"{results['mean']['R**2']:2.2f} ± {results['sem']['R**2']:2.2f}", end="")
    print("\n", end="")
    print(f"{results['mean']['slope']:2.2f} ± {results['sem']['slope']:2.2f}", end="")
    print("\n", end="")
    print(f"{results['mean']['intercept']:2.2f} ± {results['sem']['intercept']:2.2f}", end="")
    print("\n")

# Binding free energy

(Create statistics files for re-use)

In [102]:
from multiprocessing import Pool

In [114]:
with Pool(12) as p:

    p.starmap(
        write_statistics,
        [
            (experimental, smirnoff_combined, "G", "experimental_smirnoff"),
            (experimental, bgbg_combined, "G", "experimental_bgbg"),
            (experimental, bg2bg2_combined, "G", "experimental_bg2bg2"),
            (bgbg_combined, smirnoff_combined, "G", "bgbg_smirnoff"),
            (bg2bg2_combined, smirnoff_combined, "G", "bg2bg2_smirnoff"),
            (bgbg_combined, bg2bg2_combined, "G", "bgbg_bg2bg2"),
            (bgbg, bg2bg2, "G", "bgbg_bg2bg2_by_orientation"),
            (bgbg, smirnoff, "G", "bgbg_smirnoff_by_orientation"),
            (bg2bg2, smirnoff, "G", "bg2bg2_smirnoff_by_orientation"),
        ],
    )


Skipping results/experimental_bg2bg2_dG_statistics_overall.csv...
Skipping results/experimental_bgbg_dG_statistics_overall.csv...
Skipping results/experimental_bgbg_dG_statistics_aliphatic_ammoniums.csv...
Skipping results/experimental_bg2bg2_dG_statistics_aliphatic_ammoniums.csv...
Skipping results/experimental_bgbg_dG_statistics_cyclic_alcohols.csv...
Skipping results/experimental_smirnoff_dG_statistics_overall.csv...
Skipping results/experimental_bg2bg2_dG_statistics_cyclic_alcohols.csv...
Skipping results/experimental_bgbg_dG_statistics_aliphatic_carboxylates.csv...
Skipping results/experimental_smirnoff_dG_statistics_aliphatic_ammoniums.csv...
Skipping results/experimental_bg2bg2_dG_statistics_aliphatic_carboxylates.csv...
Skipping results/experimental_smirnoff_dG_statistics_cyclic_alcohols.csv...
Skipping results/experimental_smirnoff_dG_statistics_aliphatic_carboxylates.csv...
Skipping results/bgbg_bg2bg2_by_orientation_dG_statistics_overall.csv...
Skipping results/bgbg_bg2bg2_d

## Enthalpy

In [115]:
with Pool(12) as p:

    p.starmap(
        write_statistics,
        [
            (experimental, smirnoff_combined, "H", "experimental_smirnoff"),
            (experimental, bgbg_combined, "H", "experimental_bgbg"),
            (experimental, bg2bg2_combined, "H", "experimental_bg2bg2"),
            (bgbg_combined, smirnoff_combined, "H", "bgbg_smirnoff"),
            (bg2bg2_combined, smirnoff_combined, "H", "bg2bg2_smirnoff"),
            (bgbg_combined, bg2bg2_combined, "H", "bgbg_bg2bg2"),
            (bgbg, bg2bg2, "H", "bgbg_bg2bg2_by_orientation"),
            (bgbg, smirnoff, "H", "bgbg_smirnoff_by_orientation"),
            (bg2bg2, smirnoff, "H", "bg2bg2_smirnoff_by_orientation"),
        ],
    )


Skipping results/experimental_bgbg_dH_statistics_overall.csv...
Skipping results/experimental_bg2bg2_dH_statistics_overall.csv...
Skipping results/experimental_bgbg_dH_statistics_aliphatic_ammoniums.csv...
Skipping results/experimental_bg2bg2_dH_statistics_aliphatic_ammoniums.csv...
Skipping results/experimental_bg2bg2_dH_statistics_cyclic_alcohols.csv...
Skipping results/experimental_smirnoff_dH_statistics_overall.csv...
Skipping results/experimental_bgbg_dH_statistics_cyclic_alcohols.csv...
Skipping results/experimental_bg2bg2_dH_statistics_aliphatic_carboxylates.csv...
Skipping results/experimental_smirnoff_dH_statistics_aliphatic_ammoniums.csv...
Skipping results/experimental_smirnoff_dH_statistics_cyclic_alcohols.csv...
Skipping results/experimental_bgbg_dH_statistics_aliphatic_carboxylates.csv...
Skipping results/experimental_smirnoff_dH_statistics_aliphatic_carboxylates.csv...
Skipping results/bgbg_bg2bg2_dH_statistics_overall.csv...
Skipping results/bgbg_bg2bg2_dH_statistics_al

# Entropy

In [117]:
bgbg["-TdS"] = bgbg["Delta G"] - bgbg["Delta H"]
bgbg["-TdS_SEM"] = np.sqrt(bgbg["G_SEM"]**2 + bgbg["H_SEM"]**2)

bg2bg2["-TdS"] = bg2bg2["Delta G"] - bg2bg2["Delta H"]
bg2bg2["-TdS_SEM"] = np.sqrt(bg2bg2["G_SEM"]**2 + bg2bg2["H_SEM"]**2)

smirnoff["-TdS"] = smirnoff["Delta G"] - smirnoff["Delta H"]
smirnoff["-TdS_SEM"] = np.sqrt(smirnoff["G_SEM"]**2 + smirnoff["H_SEM"]**2)


smirnoff_combined["-TdS"] = smirnoff_combined["Delta G"] - smirnoff_combined["Delta H"]
smirnoff_combined["-TdS_SEM"] = np.sqrt(smirnoff_combined["G_SEM"]**2 + smirnoff_combined["H_SEM"]**2)

bgbg_combined["-TdS"] = bgbg_combined["Delta G"] - bgbg_combined["Delta H"]
bgbg_combined["-TdS_SEM"] = np.sqrt(bgbg_combined["G_SEM"]**2 + bgbg_combined["H_SEM"]**2)

bg2bg2_combined["-TdS"] = bg2bg2_combined["Delta G"] - bg2bg2_combined["Delta H"]
bg2bg2_combined["-TdS_SEM"] = np.sqrt(bg2bg2_combined["G_SEM"]**2 + bg2bg2_combined["H_SEM"]**2)

experimental["-TdS"] = experimental["Delta G"] - experimental["Delta H"]
experimental["-TdS_SEM"] = np.sqrt(experimental["G_SEM"]**2 + experimental["H_SEM"]**2)

In [93]:
from summarize_statistics import write_entropy_statistics

In [120]:
with Pool(12) as p:

    p.starmap(
        write_entropy_statistics,
        [
            (experimental, smirnoff_combined, "-TdS", "experimental_smirnoff"),
            (experimental, bgbg_combined, "-TdS", "experimental_bgbg"),
            (experimental, bg2bg2_combined, "-TdS", "experimental_bg2bg2"),
            (bgbg_combined, smirnoff_combined, "-TdS", "bgbg_smirnoff"),
            (bg2bg2_combined, smirnoff_combined, "-TdS", "bg2bg2_smirnoff"),
            (bgbg_combined, bg2bg2_combined, "-TdS", "bgbg_bg2bg2"),
            (bgbg, bg2bg2, "-TdS", "bgbg_bg2bg2_by_orientation"),
            (bgbg, smirnoff, "-TdS", "bgbg_smirnoff_by_orientation"),
            (bg2bg2, smirnoff, "-TdS", "bg2bg2_smirnoff_by_orientation"),
        ],
    )


Skipping results/experimental_bg2bg2_-TdS_statistics_overall.csv...
Skipping results/experimental_bgbg_-TdS_statistics_overall.csv...
Skipping results/experimental_bg2bg2_-TdS_statistics_aliphatic_ammoniums.csv...
Skipping results/experimental_bgbg_-TdS_statistics_aliphatic_ammoniums.csv...
Skipping results/experimental_bgbg_-TdS_statistics_cyclic_alcohols.csv...
Skipping results/bgbg_bg2bg2_by_orientation_-TdS_statistics_overall.csv...
Skipping results/experimental_bgbg_-TdS_statistics_aliphatic_carboxylates.csv...
Skipping results/bgbg_bg2bg2_by_orientation_-TdS_statistics_aliphatic_ammoniums.csv...
Skipping results/experimental_smirnoff_-TdS_statistics_overall.csv...
Skipping results/experimental_bg2bg2_-TdS_statistics_cyclic_alcohols.csv...
Skipping results/bgbg_smirnoff_-TdS_statistics_overall.csv...
Skipping results/bgbg_bg2bg2_by_orientation_-TdS_statistics_cyclic_alcohols.csv...
Skipping results/experimental_bg2bg2_-TdS_statistics_aliphatic_carboxylates.csv...
Skipping results/

In [95]:
from multiprocessing import Pool

In [100]:
with Pool(8) as p:
    
    p.starmap(write_entropy_statistics, 
                [(bgbg, bg2bg2, "-TdS", "bgbg_bg2bg2_by_orientation"), 
                 (bgbg, smirnoff, "-TdS", "bgbg_smirnoff_by_orientation"), 
                 (bg2bg2, smirnoff, "-TdS", "bg2bg2_smirnoff_by_orientation")])

Skipping results/bgbg_bg2bg2_by_orientation_-TdS_statistics_overall.csv...


Process ForkPoolWorker-25:
Process ForkPoolWorker-26:
Process ForkPoolWorker-27:
Process ForkPoolWorker-31:
Process ForkPoolWorker-30:
Process ForkPoolWorker-28:
Process ForkPoolWorker-32:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/dslochower/data/applications/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/dslochower/data/applications/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/dslochower/data/applications/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/dslochower/data/applications/anaconda3/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/dslochower/data/applicat

KeyboardInterrupt: 

In [84]:
from make_statistics_table import table

In [88]:
table(thermodynamic_quantity="G")

SMIRNOFF99Frosst	0.91	0.11	-0.01	0.14	0.34	0.11	0.49	0.12	-1.55	0.38	

GAFF v1.7	0.88	0.09	0.46	0.12	0.54	0.10	0.69	0.11	-0.48	0.35	

GAFF v2.1	1.68	0.09	-1.56	0.10	0.82	0.08	1.19	0.09	-1.00	0.28	



In [89]:
table(thermodynamic_quantity="H")

SMIRNOFF99Frosst	1.85	0.23	0.77	0.26	0.44	0.12	0.85	0.17	0.41	0.52	

GAFF v1.7	2.54	0.24	1.84	0.27	0.39	0.12	0.80	0.18	1.36	0.60	

GAFF v2.1	2.21	0.23	-1.64	0.23	0.75	0.08	1.38	0.12	-0.69	0.40	



In [119]:
experimental

,Unnamed: 0,System,Delta G,G_SEM,Delta H,H_SEM,-TdS,-TdS_SEM
0,0,a-bam,-1.58,0.02,-2.17,0.05,0.59,0.053852
1,1,a-nmb,-1.69,0.02,-2.57,0.06,0.88,0.063246
2,2,a-mba,-1.76,0.02,-2.68,0.07,0.92,0.072801
3,3,a-pam,-2.72,0.00,-3.28,0.02,0.56,0.020000
4,4,a-ham,-3.53,0.00,-4.19,0.02,0.66,0.020000
5,5,a-nmh,-3.52,0.01,-4.20,0.08,0.68,0.080623
6,6,a-mha,-3.60,0.00,-4.28,0.02,0.68,0.020000
7,7,a-hpa,-4.14,0.00,-4.66,0.02,0.52,0.020000
8,8,a-mhp,-4.17,0.00,-4.74,0.02,0.57,0.020000
9,9,a-oam,-4.61,0.01,-5.46,0.03,0.85,0.031623
